$\newcommand{\To}{\Rightarrow}$

In [1]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [17]:
from kernel.type import TVar
from kernel.term import Term, Var
from logic.nat import natT, zero, one
from logic import basic
from syntax import printer

thy = basic.loadTheory('nat')

## Substitution

Substitution on terms is analogous to substitution on types, but more complicated due to the presence of abstractions (lambda terms), and the need to substitute for both type and term variables. Consider the following term:

In [6]:
Ta = TVar("a")
a = Var("a", Ta)
b = Var("b", Ta)
t = Term.mk_equals(a, b)
print(repr(t))

Comb(Comb(Const(equals,'a => 'a => bool),Var(a,'a)),Var(b,'a))


Observe that there are both type and term variables in $t$. The method `subst_type` takes a dictionary of assignments for type variables, and substitutes using this dictionary.

In [11]:
t2 = t.subst_type({"a": natT})
print(repr(t2))

Comb(Comb(Const(equals,nat => nat => bool),Var(a,nat)),Var(b,nat))


Next, we can apply the method `subst`, which takes a dictionary of assignments for (term) variables, and substitutes using this dictionary.

In [19]:
t3 = t2.subst({"a": zero, "b": one})
print(printer.print_term(thy, t3))

0 = 1
